# Statistics of star measurements

The purpose of this notebook is to estimate what fraction of stars in a given field have measurements in one, two and three filters. 

To get a straight forward estimate we use the field index from the field's crossmatch table, as this indicates each star's ID in the reference images for each filter.  We will use this as a rough indicator of whether the star would have produced valid lightcurve 
measurements in that filter. 

In [8]:
from pyDANDIA import crossmatch
import numpy as np

We will use the data from ROME-FIELD-01 as a proxy, since it should be reasonably consistent with the other fields. 

In [3]:
xmatch_file = '/Users/rstreet/ROME/ROME-FIELD-01/DR1/ROME-FIELD-01_field_crossmatch.fits'

xmatch = crossmatch.CrossMatchTable()
xmatch.load(xmatch_file,log=None)

In [4]:
xmatch.field_index

field_id,ra,dec,quadrant,quadrant_id,gaia_source_id,ROME-FIELD-01_lsc-doma-1m0-05-fa15_ip_index,ROME-FIELD-01_lsc-doma-1m0-05-fa15_rp_index,ROME-FIELD-01_lsc-doma-1m0-05-fa15_gp_index,ROME-FIELD-01_lsc-domb-1m0-09-fa03_ip_index,ROME-FIELD-01_coj-domb-1m0-03-fa11_gp_index,ROME-FIELD-01_coj-domb-1m0-03-fa11_rp_index,ROME-FIELD-01_coj-domb-1m0-03-fa11_ip_index,ROME-FIELD-01_coj-doma-1m0-11-fa12_gp_index,ROME-FIELD-01_coj-doma-1m0-11-fa12_rp_index,ROME-FIELD-01_coj-doma-1m0-11-fa12_ip_index,ROME-FIELD-01_cpt-doma-1m0-10-fa16_gp_index,ROME-FIELD-01_cpt-doma-1m0-10-fa16_rp_index,ROME-FIELD-01_cpt-doma-1m0-10-fa16_ip_index,ROME-FIELD-01_cpt-domb-1m0-13-fa14_gp_index,ROME-FIELD-01_cpt-domb-1m0-13-fa14_rp_index,ROME-FIELD-01_cpt-domb-1m0-13-fa14_ip_index,ROME-FIELD-01_cpt-domc-1m0-12-fa06_rp_index,ROME-FIELD-01_cpt-domc-1m0-12-fa06_ip_index,ROME-FIELD-01_cpt-domc-1m0-12-fa06_gp_index
int64,float64,float64,int64,int64,str19,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
1,267.61861696019145,-29.829605383706895,4,1,4056436121079692032,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,267.70228408545813,-29.83032824102953,4,2,4056435567040767488,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,267.9873108673885,-29.829734325692858,3,1,4056444539267431040,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,267.9585073984874,-29.83002538112054,3,2,4056444878525743616,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,267.9623466389135,-29.82994179424344,3,3,4056444917224603904,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,267.943683356543,-29.830113202355186,3,4,4056444951538372736,6,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,267.90449275089594,-29.830465810573223,3,5,4056445084668331520,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,267.9504950018423,-29.830247462548577,3,6,4056444951538344320,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,267.9778110411362,-29.83012645385565,3,7,4056444156948804608,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Using the star indices for the primary reference datasets, we can count the number of stars that were detected in all 3 passbands.

In [14]:
# Column names for the indices of each star in the primary reference datasets for each passband.
ref_col_g = 'ROME-FIELD-01_lsc-doma-1m0-05-fa15_gp_index'
ref_col_r = 'ROME-FIELD-01_lsc-doma-1m0-05-fa15_rp_index'
ref_col_i = 'ROME-FIELD-01_lsc-doma-1m0-05-fa15_ip_index'

# Count the number of stars with non-zero indices in all three columns, indicating that the star was 
# detected in all three reference images
#mask = (xmatch.field_index[ref_col_g] > 0) and (xmatch.field_index[ref_col_r] > 0) and (xmatch.field_index[ref_col_i] > 0)
gdx = np.where(xmatch.field_index[ref_col_g] > 0)[0]
rdx = np.where(xmatch.field_index[ref_col_r] > 0)[0]
idx = np.where(xmatch.field_index[ref_col_i] > 0)[0]

mask = list((set(gdx).intersection(set(rdx))).intersection(set(idx)))
n3bands = len(mask)
n3percent = (n3bands/len(xmatch.field_index))*100.0
print('Number of stars measured in all three passbands=' + str(n3bands) + ', percentage of total=' + str(n3percent) + '%')

Number of stars measured in all three passbands=73736, percentage of total=18.280172349677464%


Counting the number detected in two passbands requires us to handle cases of (i,g), (r,g) and (i,r).

In [19]:
# Filter pairs are defined in set order, with the last column in the tuple being the column where the star is NOT measured. 
filter_sets = {
    'g,i': (ref_col_g, ref_col_i, ref_col_r),
    'r,g': (ref_col_r, ref_col_g, ref_col_i),
    'i,r': (ref_col_i, ref_col_r, ref_col_g)
}

# Counting the number of stars measured in the two selected bands, accumulating the total percentage:
n2bands = 0
for label, fset in filter_sets.items():
    idx1 = np.where(xmatch.field_index[fset[0]] > 0)[0]
    idx2 = np.where(xmatch.field_index[fset[1]] > 0)[0]
    idx3 = np.where(xmatch.field_index[fset[2]] == 0)[0]
    mask = list((set(idx1).intersection(set(idx2))).intersection(set(idx3)))
    nbands = len(mask)
    n2bands += nbands
    npercent = (nbands/len(xmatch.field_index))*100.0
    print('Number of stars measured in passbands ' + label + '=' + str(nbands) + ', percentage of total=' + str(npercent) + '%')

n2percent = (n2bands/len(xmatch.field_index))*100.0
print('Total number of stars measured in two passbands=' + str(n2bands) + ', percentage of total=' + str(n2percent) + '%')

Number of stars measured in passbands g,i=4093, percentage of total=1.0147112052081733%
Number of stars measured in passbands r,g=2509, percentage of total=0.6220157375683623%
Number of stars measured in passbands i,r=89024, percentage of total=22.070278605534426%
Total number of stars measured in two passbands=95626, percentage of total=23.707005548310963%


Lastly, counting the number of stars with measurements in only one passband. 

In [20]:
# Filter in set order, with the last two columns in the tuple being the columns where the star is NOT measured. 
filter_sets = {
    'g only': (ref_col_g, ref_col_i, ref_col_r),
    'r only': (ref_col_r, ref_col_g, ref_col_i),
    'i only': (ref_col_i, ref_col_r, ref_col_g)
}

# Counting the number of stars measured in the two selected bands, accumulating the total percentage:
n1band = 0
for label, fset in filter_sets.items():
    idx1 = np.where(xmatch.field_index[fset[0]] > 0)[0]
    idx2 = np.where(xmatch.field_index[fset[1]] == 0)[0]
    idx3 = np.where(xmatch.field_index[fset[2]] == 0)[0]
    mask = list((set(idx1).intersection(set(idx2))).intersection(set(idx3)))
    nbands = len(mask)
    n1band += nbands
    npercent = (nbands/len(xmatch.field_index))*100.0
    print('Number of stars measured in ' + label + '=' + str(nbands) + ', percentage of total=' + str(npercent) + '%')

n1percent = (n1band/len(xmatch.field_index))*100.0
print('Total number of stars measured in one passband only=' + str(n1band) + ', percentage of total=' + str(n1percent) + '%')

Number of stars measured in g only=4743, percentage of total=1.1758551786714795%
Number of stars measured in r only=10567, percentage of total=2.619705180902704%
Number of stars measured in i only=59289, percentage of total=14.698561604101487%
Total number of stars measured in one passband only=74599, percentage of total=18.49412196367567%
